In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [3]:
# 비디오 파일 이름
input_file = "D:\\Pitch\\sample_videos\\14_left.mp4" # 여기에 비디오 경로 넣으면 됩니다!

# 비디오 파일의 경우 이것을 사용하세요:
cap = cv2.VideoCapture(input_file)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [4]:
# 출력 파일 경로 및 이름 설정
output_file = os.path.splitext(input_file)[0] + "_dotted.mp4"
out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

In [5]:
# 손목 관절 좌표화 코드
right_wrist_coordinates = [] # 우완투수

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("비디오 파일을 처리하는데 실패했습니다.")
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        image_height, image_width, _ = image.shape

        if results.pose_landmarks:
            right_wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            cv2.circle(image, (int(right_wrist.x * image_width), int(right_wrist.y * image_height)), 5, (0, 0, 255), -1)
            right_wrist_coordinates.append((right_wrist.x, right_wrist.y))

        out.write(image)

        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
out.release()  # 출력 파일 닫기
cv2.destroyAllWindows()

비디오 파일을 처리하는데 실패했습니다.


In [6]:
# 팔꿈치 좌표 출력
for i, coord in enumerate(right_wrist_coordinates):
    print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

Frame 1: x = 801.1463284492493, y = 970.6473541259766
Frame 2: x = 792.0591115951538, y = 963.0441284179688
Frame 3: x = 792.0072913169861, y = 962.5518035888672
Frame 4: x = 789.6668815612793, y = 962.2525405883789
Frame 5: x = 781.715075969696, y = 959.3342971801758
Frame 6: x = 781.0506820678711, y = 960.4887771606445
Frame 7: x = 779.4127750396729, y = 961.1842346191406
Frame 8: x = 780.2119016647339, y = 961.5109634399414
Frame 9: x = 781.3689422607422, y = 961.864013671875
Frame 10: x = 781.5786051750183, y = 963.2881164550781
Frame 11: x = 781.9039463996887, y = 963.1948471069336
Frame 12: x = 781.9866013526917, y = 963.9284133911133
Frame 13: x = 781.1759519577026, y = 963.9525604248047
Frame 14: x = 781.4339590072632, y = 963.9665222167969
Frame 15: x = 782.1803641319275, y = 964.0125274658203
Frame 16: x = 782.6836967468262, y = 963.259391784668
Frame 17: x = 782.8318190574646, y = 958.6376953125
Frame 18: x = 782.9294729232788, y = 958.853702545166
Frame 19: x = 784.32237625

In [8]:
# 좌표를 데이터프레임에 저장
import pandas as pd
df = pd.DataFrame(right_wrist_coordinates, columns=["x", "y"]) # x_back, x_left, x_right, y_back, y_left, y_right 중 선택
df.head()

,x,y
0,0.741802,0.505545
1,0.733388,0.501585
2,0.733340,0.501329
3,0.731173,0.501173
4,0.723810,0.499653


In [9]:
output_dir = "D:/Pitch/coordinates"
output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_wrist_coordinates.csv"
output_file_path = os.path.join(output_dir, output_filename)

df.to_csv(output_file_path, index=False)

In [8]:
import cv2

cap = cv2.VideoCapture(output_file)

if (cap.isOpened()== False): 
    print("비디오를 열 수 없습니다.")
  
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('Frame', frame)
        # q 키를 눌러 비디오 재생을 중지합니다.
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()

# q 누르면 비디오 중지됩니다!!! 제발!!!